In [ ]:
# Import the necessary packages and seet seed for reproducibility .

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

# Expert Module

In [ ]:
class Expert(nn.Module):
  """ An MLP is a simple linear layer followed by a non-linearity i.e each Expert   """

  def __init__(self,n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd,4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd,n_embd),
        nn.Dropout(dropout),
    )

  def forward(self,x):
    return self.net(x)

In [ ]:
!wget https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/input.txt

--2025-06-30 17:48:35--  https://raw.githubusercontent.com/AviSoori1x/makeMoE/main/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  --.-KB/s    in 0.005s  

2025-06-30 17:48:35 (206 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



# Step 2: Implement a Router

In [ ]:
# Understanding how gating works

num_experts = 4
top_k= 2
n_embed = 32

# Example multi-head attention output for a simple illustrative example, consider n_embed=32, context_length = 4
mh_output = torch.randn(1,4,n_embed)

topkgate_linear = nn.Linear(n_embed, num_experts) # nn.Linear(32,4)

logits = topkgate_linear(mh_output)

print(logits)

tensor([[[-0.8934,  0.1072,  0.5144, -0.2811],
         [-0.8497, -0.9384, -0.1564, -0.3949],
         [-0.1428,  0.6368,  0.3035,  0.9877],
         [-0.3160, -0.8139, -0.3333, -0.0203]]], grad_fn=<ViewBackward0>)


# Step 3: Implement topk load balancing

In [ ]:
top_k_logits, top_k_indices = logits.topk(top_k,dim=-1) # Get top-k experts
top_k_logits, top_k_indices

(tensor([[[ 0.5144,  0.1072],
          [-0.1564, -0.3949],
          [ 0.9877,  0.6368],
          [-0.0203, -0.3160]]], grad_fn=<TopkBackward0>),
 tensor([[[2, 1],
          [2, 3],
          [3, 1],
          [3, 0]]]))

# Step 4 : Use -inf and apply softmax


In [ ]:
zeros = torch.full_like(logits,float('-inf')) # full_like clones a tensor and fills it with a specified value
sparse_logits = zeros.scatter(-1,top_k_indices,top_k_logits)
sparse_logits

tensor([[[   -inf,  0.1072,  0.5144,    -inf],
         [   -inf,    -inf, -0.1564, -0.3949],
         [   -inf,  0.6368,    -inf,  0.9877],
         [-0.3160,    -inf,    -inf, -0.0203]]], grad_fn=<ScatterBackward0>)

In [ ]:
gating_output = F.softmax(sparse_logits,dim=-1)
gating_output

tensor([[[0.0000, 0.3996, 0.6004, 0.0000],
         [0.0000, 0.0000, 0.5594, 0.4406],
         [0.0000, 0.4132, 0.0000, 0.5868],
         [0.4266, 0.0000, 0.0000, 0.5734]]], grad_fn=<SoftmaxBackward0>)

# Step 5: Create a class for TopKRouting

In [ ]:
# First define the top k router module
class TopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(TopkRouter, self).__init__()
        self.top_k = top_k
        self.linear =nn.Linear(n_embed, num_experts)

    def forward(self, mh_ouput):
        # mh_ouput is the output tensor from multihead self attention block
        logits = self.linear(mh_output)
        top_k_logits, indices = logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices


In [ ]:
#Testing this out:
num_experts = 4
top_k = 2
n_embd = 32

mh_output = torch.randn(2, 4, n_embd)  # Example input
top_k_gate = TopkRouter(n_embd, num_experts, top_k)
gating_output, indices = top_k_gate(mh_output)
gating_output.shape, gating_output, indices


(torch.Size([2, 4, 4]),
 tensor([[[0.0000, 0.0000, 0.5321, 0.4679],
          [0.0000, 0.8659, 0.1341, 0.0000],
          [0.0000, 0.4096, 0.0000, 0.5904],
          [0.0000, 0.0000, 0.5828, 0.4172]],
 
         [[0.6210, 0.0000, 0.3790, 0.0000],
          [0.7005, 0.0000, 0.2995, 0.0000],
          [0.0000, 0.8630, 0.1370, 0.0000],
          [0.0000, 0.3759, 0.6241, 0.0000]]], grad_fn=<SoftmaxBackward0>),
 tensor([[[2, 3],
          [1, 2],
          [3, 1],
          [2, 3]],
 
         [[0, 2],
          [0, 2],
          [1, 2],
          [2, 1]]]))

# Step 6 Create. a class for Noisy Topk Routing

Noisy top-k Gating is an important tool in training MoE models.

Essentially, you don't want all the tokens to be sent to the same set of 'favored' experts.

You want a fine balance of exploitation and exploraation. For this purpose, to load balance, it is helps logits from the gating linear layer. This makes trainning more efficient

In [ ]:
#Changing the above to accomodate noisy top-k gating
class NoisyTopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(NoisyTopkRouter, self).__init__()
        self.top_k = top_k
        #layer for router logits
        self.topkroute_linear = nn.Linear(n_embed, num_experts)
        self.noise_linear =nn.Linear(n_embed, num_experts)


    def forward(self, mh_output):
        # mh_ouput is the output tensor from multihead self attention block
        logits = self.topkroute_linear(mh_output)

        #Noise logits
        noise_logits = self.noise_linear(mh_output)

        #Adding scaled unit gaussian noise to the logits
        noise = torch.randn_like(logits)*F.softplus(noise_logits)
        noisy_logits = logits + noise

        top_k_logits, indices = noisy_logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(noisy_logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices


In [ ]:
#Testing this out, again:
num_experts = 8
top_k = 2
n_embd = 16

mh_output = torch.randn(2, 4, n_embd)  # Example input
noisy_top_k_gate = NoisyTopkRouter(n_embd, num_experts, top_k)
gating_output, indices = noisy_top_k_gate(mh_output)
gating_output.shape, gating_output, indices



(torch.Size([2, 4, 8]),
 tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6640, 0.3360],
          [0.4700, 0.0000, 0.5300, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.4277, 0.0000, 0.0000, 0.0000, 0.5723, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.4105, 0.0000, 0.0000, 0.5895]],
 
         [[0.0000, 0.4177, 0.0000, 0.0000, 0.5823, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.6713, 0.0000, 0.0000, 0.3287, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.1670, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8330],
          [0.4322, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5678]]],
        grad_fn=<SoftmaxBackward0>),
 tensor([[[6, 7],
          [2, 0],
          [6, 2],
          [7, 4]],
 
         [[4, 1],
          [1, 4],
          [7, 1],
          [7, 0]]]))

In [ ]:
class SparseMoE(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(SparseMoE, self).__init__()
        self.router = NoisyTopkRouter(n_embed, num_experts, top_k)
        self.experts = nn.ModuleList([Expert(n_embed) for _ in range(num_experts)])
        self.top_k = top_k

    def forward(self, x):
        gating_output, indices = self.router(x)
        final_output = torch.zeros_like(x)

        # Reshape inputs for batch processing
        flat_x = x.view(-1, x.size(-1))
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))

        # Process each expert in parallel
        for i, expert in enumerate(self.experts):
            # Create a mask for the inputs where the current expert is in top-k
            expert_mask = (indices == i).any(dim=-1)
            flat_mask = expert_mask.view(-1)

            if flat_mask.any():
                expert_input = flat_x[flat_mask]
                expert_output = expert(expert_input)

                # Extract and apply gating scores
                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)
                weighted_output = expert_output * gating_scores

                # Update final output additively by indexing and adding
                final_output[expert_mask] += weighted_output.squeeze(1)

        return final_output


In [ ]:
import torch
import torch.nn as nn

#Let's test this out
num_experts = 8
top_k = 2
n_embd = 16
dropout=0.1

mh_output = torch.randn(1, 4, n_embd)  # Example multi-head attention output
sparse_moe = SparseMoE(n_embd, num_experts, top_k)
final_output = sparse_moe(mh_output)
print("Shape of the final output:", final_output.shape)
print(final_output)


Shape of the final output: torch.Size([1, 4, 16])
tensor([[[ 0.4342, -0.4548, -0.0086, -0.0395, -0.1140, -0.0925, -0.2745,
          -0.0195, -0.3261,  0.1218, -0.0654,  0.1763, -0.1743, -0.2058,
          -0.1101,  0.1939],
         [-0.0275, -0.1477, -0.0534,  0.4124,  0.4952, -0.4857, -0.2878,
          -0.1663,  0.3566,  0.3007,  0.0182,  0.4281,  0.2393,  0.4560,
          -0.1753,  0.0000],
         [-0.0070,  0.0822,  0.0469,  0.2413, -0.3832, -0.0366, -0.1590,
          -0.1679, -0.2089,  0.0903,  0.2140,  0.2162, -0.0403, -0.0554,
          -0.1221,  0.2234],
         [-0.1403, -0.1466,  0.3349, -0.1171,  0.8054,  0.3437, -0.2110,
           0.2405, -0.0367,  0.3583, -0.0914, -0.3995, -0.4149, -0.0041,
           0.0000, -0.0971]]], grad_fn=<IndexPutBackward0>)


# Step 8: Putting together all teh building blocks of MoE

In [ ]:
#Create a self attention + mixture of experts block, that may be repeated several number of times
class Block(nn.Module):
    """ Mixture of Experts Transformer block: communication followed by computation (multi-head self attention + SparseMoE) """

    def __init__(self, n_embed, n_head, num_experts, top_k):
        # n_embed: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.smoe = SparseMoE(n_embed, num_experts, top_k)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.smoe(self.ln2(x))
        return x


In [ ]:
class SparseMoELanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head, num_experts=num_experts,top_k=top_k) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


# Step 9: Code the entire transformer block: (Multi-Head Attention)

In [ ]:
class Head(nn.Module):
  """ one head of self-attention """

  def __init__(self,head_size):
    super().__init__()
    self.key = nn.Linear(n_embed,head_size,bias=False)
    self.query = nn.Linear(n_embed,head_size,bias=False)
    self.value = nn.Linear(n_embed,head_size,bias=False)

    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
    self.dropout = nn.Dropout(dropout)



  def forward(self,x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    # Computer attention scores ("affinities")
    wei = q @ k.transpose(-2,-1)*C**-0.5 # (B,T,C) @ (B,C,T) -> (B,T,T)
    wei = wei.masked_fill(self.tril[:T,:T]== 0,float('-inf')) # (B,T,T)
    wei = F.softmax(wei,dim=-1)
    wei = self.dropout(wei)

    # perform the weighted aggregation ofthe values
    v = self.value(x) # (B,T,C)
    out = wei @ v # (B,T,T) @ (B,T,C) -> (B,T,C)
    return out


# Multi-Headed Self Attention

class MultiHeadAttention(nn.Module):
  """ multiple heads of self-attention in parallel """


  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embed,n_embed)
    self.dropout = nn.Dropout(dropout)


  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out




# Step 10 Code the entire transfomer block: (Assemble all layers)

Start by defining a Self-Attention combined with Mixture-of-Experts (MoE) block, designed to be modular and reusable across multiple layers of the model.
(For better clarity, key architectural parameters are copied and reused.)

# Step 11: Define entire language model architecture


Finally putting it all together to crease a sparse mixture of experts language model

In [ ]:
class SparseMoELanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head, num_experts=num_experts,top_k=top_k) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


# Step 12: Create training and testing data

In [ ]:
torch.manual_seed(42)

with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

# here are all the unique characters that occur in this text

chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string,output a list of integers
decode = lambda l: ''.join(itos[i] for i in l)  # # decoder: take a list of integers, output a string


# Train and test splits

data = torch.tensor(encode(text),dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading

def get_batch(split):
    # generate a small batch of data of inputs x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # Input: tokens at positions [i, i+1, ..., i+block_size-1]
    x = torch.stack([data[i:i+block_size] for i in ix])

    # Target: tokens at positions [i+1, i+2, ..., i+block_size]
    # This is the "next token" for each input token
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])

    x, y = x.to(device), y.to(device)
    return x, y





# Step 13: Define LLM Loss

In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


# Step 14: Define training Loop parameters and other hyperparameters

In [ ]:
# First defining hyperparameters and boiler place code. Imports and data preparation code is repeated for convenience

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import init

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context lenght for prediction?
max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 400
head_size = 16
n_embed = 128
n_head = 8
n_layer = 8
dropout = 0.1
num_experts = 8
top_k = 2

# Step 15: Initialize the entire model

In [ ]:
def kaiming_init__weights(m):
  if isinstance (m, (nn.Linear)):
    init.kaiming_normal_(m.weight)

In [ ]:
model = SparseMoELanguageModel()
model.apply(kaiming_init__weights)

SparseMoELanguageModel(
  (token_embedding_table): Embedding(65, 128)
  (position_embedding_table): Embedding(32, 128)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=128, out_features=16, bias=False)
            (query): Linear(in_features=128, out_features=16, bias=False)
            (value): Linear(in_features=128, out_features=16, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (smoe): SparseMoE(
        (router): NoisyTopkRouter(
          (topkroute_linear): Linear(in_features=128, out_features=8, bias=True)
          (noise_linear): Linear(in_features=128, out_features=8, bias=True)
        )
        (experts): ModuleList(
          (0-7): 8 x Expert(
            (net): Sequential(
             

# Step 16: Run the pre-training Loop

In [ ]:
m = model.to(device)
# Print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6,'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print("step {}: train loss {:.4f}, val loss {:.4f}".format(iter, losses['train'], losses['val']))

  # sample. a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = model(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

# Debug the vocabulary mismatch
print("=== VOCABULARY DEBUG ===")

# Check actual vocabulary
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
actual_vocab_size = len(chars)

print(f"Actual vocabulary size from text: {actual_vocab_size}")
print(f"Current vocab_size variable: {vocab_size}")

# Check model's expected vocabulary size
model_vocab_size = model.lm_head.out_features
print(f"Model's output layer expects: {model_vocab_size}")

# Show the mismatch
if actual_vocab_size != model_vocab_size:
    print(f"MISMATCH: Model expects {model_vocab_size} but data has {actual_vocab_size}")
    print("Solution: Recreate the model with the correct vocab_size")
else:
    print("Vocabulary sizes match")

print("=== END DEBUG ===")

8.996545 M parameters
step 0: train loss 5.1127, val loss 5.1287
step 100: train loss 2.6564, val loss 2.6589
step 200: train loss 2.4973, val loss 2.4938
step 300: train loss 2.3931, val loss 2.3948
step 400: train loss 2.3098, val loss 2.3223
step 500: train loss 2.2309, val loss 2.2536
step 600: train loss 2.1587, val loss 2.2019
step 700: train loss 2.1083, val loss 2.1624
step 800: train loss 2.0612, val loss 2.1126
step 900: train loss 2.0206, val loss 2.0880
step 999: train loss 1.9632, val loss 2.0509
=== VOCABULARY DEBUG ===
Actual vocabulary size from text: 65
Current vocab_size variable: 65
Model's output layer expects: 65
Vocabulary sizes match
=== END DEBUG ===


# Step 17: Inference

In [ ]:
# generate from the model

context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context,max_new_tokens=2000)[0].tolist))